This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'yx5K2NYbxW3VBcy3jHJf'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2014-01-01&end_date=2014-01-01&api_key=yx5K2NYbxW3VBcy3jHJf')


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data_sample = r.json()
data_sample

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2014-01-01',
  'end_date': '2014-01-01',
  'frequency': 'daily',
  'data': [],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
req = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=yx5K2NYbxW3VBcy3jHJf')


In [6]:
#create dictionary from json 
json_data = req.json()
json_data['dataset_data']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [7]:
#initialize lists
open_list = []
high_list = []
low_list = []
close_list = []
trading_volume_list = []

for i in json_data['dataset_data']['data']:
    open_list.append(i[1])
    high_list.append(i[2])
    low_list.append(i[3])
    close_list.append(i[4])
    trading_volume_list.append(i[6])


## Highest and Lowest Opening Prices

In [8]:
#open_list has some None inputs  
#let's replace them with the mean of the existing values so the min, max, and mean don't change
#find mean

open_list_sum = 0.0
count = 0
for i in open_list:
    if i is not None:
        open_list_sum += i
        count +=1

open_list_mean = open_list_sum / count

In [9]:
#replace None values with the open_list_mean

for i in range(len(open_list)):
    if open_list[i] is None:
        open_list[i] = open_list_mean
        

In [27]:
#find the highest and lowest opening prices
print('HIGHEST =', max(open_list))
print('LOWEST =', min(open_list))

HIGHEST = 53.11
LOWEST = 34.0


## Largest Change Based on High and Low Price

In [11]:
#create a list of the daily differences between the high and the low price
change_list = []
for i in range(len(high_list)):
    change_list.append(high_list[i] - low_list[i])

In [28]:
#find the max entry in change_list
print('Largest Daily Change =', round(max(change_list),2))

Largest Daily Change = 2.81


## Largest change between any two days (based on closing price)

In [13]:
#make a list for the difference between daily closing prices
day_change_list = []
for i in range(len(close_list)-1):
    day_change_list.append(close_list[i+1] - close_list[i])

In [29]:
#find the largest entry in day_change_list
print('Largest Change Between Days =', round(abs(max(day_change_list)),2))

Largest Change Between Days = 2.56


## Average Daily Trading Volume

In [30]:
mean = sum(trading_volume_list) / len(trading_volume_list)
print('Average Daily Trading Volume =', round(mean,2))

Average Daily Trading Volume = 89124.34


## Median Trading Volume

In [20]:
def calc_median(tv_list):
    result = 0
    if len(tv_list) // 2 == 0:
        result = tv_list[len(tv_list/2)] - tv_list[(len(tv_list)/2) - 1]
    else:
        result = tv_list[int((len(tv_list)-1)/2)]
    return result

In [31]:
print('Median Trading Volume =', calc_median(trading_volume_list))

Median Trading Volume = 134965.0
